# UMR QB1 - Seminar on Gene Expression Analysis

---

## Medical Problem & Research Question

### **Central Clinical Question:**
**"_What are the molecular differences between cancer cells and normal human tissue, and how can we use these differences to identify new therapeutic targets for cancer treatment?_"**

### **Why This Matters for Medicine:**
- **Cancer heterogeneity:** Different cancers have distinct molecular signatures
- **Precision medicine:** Treatments must be tailored to specific cancer types
- **Drug resistance:** Cancer cells evolve to evade treatment
- **Therapeutic targets:** New drugs are desperately needed for better patient outcomes

### **What We'll Discover:**
1. **Molecular cancer signatures:** Genes consistently altered in cancer vs normal tissue
2. **Therapeutic vulnerabilities:** Pathways that could be targeted with drugs
3. **Drug repositioning opportunities:** Existing drugs that might treat cancer
4. **Biomarker identification:** Genes that could predict treatment response

---


## Dataset: A Real Cancer vs Normal Tissue Study

**Clinical Context:** Universal Human Reference (UHR) vs Human Brain Reference (HBR)  
**Medical Relevance:** Cancer cell lines vs normal human brain tissue<br>
**Sample Size:** 6 samples (3 cancer replicates vs 3 normal brain replicates)  
**Data Type:** Paired-end RNA-sequencing (chromosome 22 subset)  
**Reference:** Griffith M, Walker JR, Spies NC, Ainscough BJ, Griffith OL (2015) Informatics for RNA Sequencing: A Web Resource for Analysis on the Cloud. *PLoS Comput Biol* 11(8): e1004393. https://doi.org/10.1371/journal.pcbi.1004393



### **Medical Samples**

#### **UHR (Universal Human Reference) = CANCER SAMPLES**
- **Composition:** Total RNA from 10 different human cancer cell lines
- **Cancer types included:** Breast, liver, cervix, testis, brain, skin cancers plus immune cells (T cell, B cell, macrophage, histocyte)
- **Why this matters:** Represents the common molecular features shared across different cancer types
- **Clinical relevance:** Helps identify pan-cancer therapeutic targets

#### **HBR (Human Brain Reference) = NORMAL TISSUE CONTROLS**
- **Composition:** Total RNA from brains of 23 healthy Caucasians, mostly 60-80 years old
- **Why brain tissue:** Provides a normal tissue baseline for comparison
- **Clinical relevance:** Shows what "healthy" gene expression looks like

### **The Biological Hypothesis:**
**Cancer cells will show systematic changes in gene expression compared to normal tissue, revealing:**
1. **Oncogenes** (cancer-promoting genes) that are overexpressed
2. **Tumor suppressors** (cancer-preventing genes) that are silenced
3. **Metabolic pathways** altered to support cancer growth
4. **Drug targets** that could selectively kill cancer cells

---

## Learning Objectives

By the end of this seminar, students will:

### **Technical Skills:**
1. Execute differential expression analysis to identify cancer biomarkers
2. Run pathway enrichment to understand cancer biology
3. Use computational drug repositioning for therapeutic discovery

### **Medical Understanding:**
1. **Interpret cancer gene signatures** in clinical context
2. **Identify potential biomarkers** for cancer diagnosis/prognosis
3. **Understand drug repositioning** as a strategy for faster therapeutic development
4. **Connect computational findings** to real cancer treatment decisions

### **Clinical Translation:**
1. **Evaluate therapeutic targets** identified through RNA-seq
2. **Assess drug candidates** for cancer treatment potential
3. **Understand precision medicine** approaches to cancer care

---

## Setup and Introduction

### Install Conda in Google Colab

After running this cell, the runtime will restart automatically. Wait for it to complete, then continue.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Verify Conda installation

In [ ]:
import condacolab
condacolab.check()

### Install all required software and packages

In [ ]:
%%bash
conda install -c bioconda -c conda-forge salmon gffread bioconductor-deseq2 r-optparse r-ggplot2 r-gprofiler2 r-dplyr r-tidyr -y -q 2>&1

### Download Data and script repository

**Background: Version Control and Reproducible Research**

In modern bioinformatics, we use version control systems like Git to manage our analysis scripts and data. This ensures reproducibility and allows collaboration. We're downloading a repository that contains:
- Pre-processed RNA-seq data files
- Analysis scripts for each step
- Reference genome files and annotations

In [ ]:
%%bash
git clone https://gitlab.uni-rostock.de/wb283/qb1rnaseq.git

In [ ]:
%%bash
ls

In [ ]:
%%bash
ls -lht

In [ ]:
# Change working directory permanently
import os
os.chdir('qb1rnaseq')

In [ ]:
%%bash
ls

In [ ]:
%%bash
tar xzf griffith-data.tar.gz

In [ ]:
%%bash
ls -lht

---

## Gene Expression

**What is RNA-seq?**
RNA-seq determines how many RNA molecules (gene expression level) were present in each sample for each gene. This process involves:

1. **Mapping reads**: Determining which gene/transcript each sequencing read came from
2. **Counting**: Tallying how many reads map to each gene
3. **Normalization**: Adjusting for sequencing depth and gene length differences


**Medical significance**

- Oncogenes (overexpressed in cancer)
- Tumor suppressors (underexpressed in cancer)
- Potential therapeutic targets

**Why use count data?**
- **Raw counts**: Represent the actual number of sequencing reads per gene
- **Statistical requirements**: Count-based statistical models (like DESeq2) need integer counts
- **Comparability**: Counts can be normalized across samples for fair comparison

---

## 1 Data


### 1.1 Dataset: Metadata and sequencings

- **UHR samples**: Mixed cancer cell lines (Universal Human Reference)
- **HBR samples**: Normal brain tissue (Human Brain Reference)
- **Chr22 subset**: Educational dataset focusing on chromosome 22 genes


In [ ]:
%%bash
cat samples.tsv

In [ ]:
%%bash
ls reads/

In [ ]:
%%bash
ls -lht

### 1.2 Gene Annotations (GTF Format)

- **Column 1**: Chromosome name (22)
- **Column 2**: Annotation source (HAVANA, ENSEMBL)
- **Column 3**: Feature type (gene, transcript, exon, CDS)
- **Column 4-5**: Start and end coordinates (1-based)
- **Column 6**: Score (usually `.` for missing)
- **Column 7**: Strand (`+` forward, `-` reverse)
- **Column 8**: Frame (for coding sequences)
- **Column 9**: Attributes (gene_id, gene_name, transcript_id, etc.)

**Medical relevance:**
- Tells us where genes are located on the chromosome
- Defines exon boundaries (coding regions)
- Links gene IDs to human-readable gene names
- Essential for quantifying expression accurately


In [ ]:
%%bash
ls refs/

In [ ]:
%%bash
# transcriptome annotations
head refs/22.gtf

### 1.3 DNA Sequence (FASTA Format)

- **Header line** (`>`): Sequence identifier and description
- **Sequence lines**: Raw DNA sequence (A, T, G, C, N)

**Biological context:**
- This is the reference genome sequence for chromosome 22
- Contains ~51 million base pairs
- Includes both coding (genes) and non-coding regions
- Used as a template to identify where RNA-seq reads originated

**How these files work together:**
1. **FASTQ files**: Contain the expression experimental data (what we measured)
2. **GTF file**: Tells us where genes are located (the map)
3. **FASTA file**: Provides the reference sequence (the genome sequence blueprint)

Together, they allow us to determine which genes the RNA-seq reads came from and quantify their expression levels.

In [ ]:
%%bash
# sequence data
head refs/22.fa

### 1.4 Gene Expression Count Matrix

**What is a count matrix?**
The count matrix is the fundamental data structure for RNA-seq analysis. It's a table where:
- **Rows**: Represent genes/transcripts
- **Columns**: Represent samples (cancer_rep1, cancer_rep2, etc.)
- **Values**: Number of sequencing reads assigned to each gene in each sample

**Why is this important?**
- **Statistical foundation**: All differential expression analysis starts with this matrix
- **Data quality**: Allows us to spot potential issues before analysis
- **Biological insight**: Shows which genes are highly vs lowly expressed
- **Clinical relevance**: Forms the basis for identifying cancer biomarkers

**Let's examine our count matrix:**


In [ ]:
%%bash
wc -l counts.tsv

In [ ]:
%%bash
head counts.tsv

In [ ]:
%%bash
tail counts.tsv

**Understanding the count matrix structure:**

**Gene_ID Column:**
- **Transcript identifiers**: From our GTF annotation file
- **ENST format**: Ensembl transcript IDs (e.g., ENST00000215832.4)
- **Version numbers**: The `.4` indicates annotation version
- **Biological meaning**: Each ID represents a specific mRNA transcript

**Sample Columns:**
- **cancer_rep1, cancer_rep2, cancer_rep3**: Three biological replicates of cancer cell lines
- **normal_rep1, normal_rep2, normal_rep3**: Three biological replicates of normal brain tissue
- **Replication importance**: Multiple replicates allow statistical testing

**Count Values:**
- **Integer numbers**: Whole read counts (0, 1, 2, 150, 5000, etc.)
- **Dynamic range**: From 0 (not expressed) to tens of thousands (highly expressed)
- **Raw counts**: Not yet normalized for library size or gene length

**What the numbers tell us:**

| Genes | High count values | Medium count values | Low count values | Zero count values |
| --- | --- | --- | --- | --- |
| Counts range | >1000 | 10-1000 | 0-10 | 0 |
| Typical content | <ul><li>Housekeeping genes: Essential cellular functions (ribosomal proteins, metabolism)</li><li>Tissue-specific genes: Genes characteristic of the tissue type</li><li>Abundant transcripts: Genes producing lots of mRNA | <ul><li>Regulated genes: Genes that may change between conditions</li><li>Functional specialization: Genes involved in specific biological processes</li><li>Potential biomarkers: Candidates for differential expression</li></ul> | <ul><li>Lowly expressed genes: Genes with minimal transcription</li><li>Tissue-inappropriate genes: Genes not typically expressed in this tissue</li><li>Technical noise: Some low counts may be background</li></ul> | <ul><li>Not expressed: Gene is turned off in this sample/condition</li><li>Below detection: Expression too low to detect reliably</li><li>Technical artifacts: Missing due to technical limitations</li></ul> |


**Clinical and Research Implications:**

**Quality indicators:**
- **Similar patterns**: Replicates should show similar count patterns
- **Dynamic range**: Healthy samples should show diverse expression levels
- **Library sizes**: Total counts should be similar across samples

**Biological insights:**
- **Condition differences**: Cancer vs normal samples may show different patterns
- **Gene expression hierarchy**: Most genes lowly expressed, few highly expressed
- **Functional categories**: Different gene types have characteristic expression levels

This count matrix serves as the foundation for identifying genes that are differentially expressed between cancer and normal tissue, which will guide our search for therapeutic targets.

## 2 Differential Expression

**What is differential expression analysis?**
Differential expression analysis identifies genes whose expression levels significantly differ between experimental conditions (cancer vs normal). This involves:

1. **Normalization**: Adjusting for differences in sequencing depth and composition
2. **Variance modeling**: Estimating biological and technical variability
3. **Statistical testing**: Using negative binomial models to test for significant differences
4. **Multiple testing correction**: Adjusting p-values for testing thousands of genes simultaneously

**How does DESeq2 work?**
DESeq2 uses sophisticated statistical methods:
- **Size factors**: Normalize for sequencing depth differences
- **Dispersion estimation**: Model gene-specific variance
- **Wald test**: Test for significant expression differences
- **Benjamini-Hochberg**: Control false discovery rate

**Key outputs:**
- **Log2 fold change**: Magnitude of expression difference
- **P-value**: Statistical significance of the difference
- **Adjusted p-value**: Corrected for multiple testing

**Clinical interpretation:**
- **Positive fold change**: Higher expression in cancer (potential oncogenes)
- **Negative fold change**: Lower expression in cancer (potential tumor suppressors)

In [ ]:
%%bash
# 2025-07-14
Rscript run_deseq2.R \
    --metadata samples.tsv \
    --expression counts.tsv \
    --output-degs degs.tsv \
    --output-plots pca_plot.png,heatmap.png,volcano_plot.png \
    --output-image deseq2_results.RData

In [ ]:
%%bash
head degs.tsv

In [ ]:
%%bash
grep "ENST00000328933.9" refs/22.gtf | cut -f9

### Data Visualization for Biological Interpretation

**Why visualize RNA-seq results?**
Visualization helps us understand complex datasets and communicate findings effectively. Our three key plots serve different purposes:

**1. PCA Plot (Principal Component Analysis):**
- **Purpose**: Shows overall similarity/differences between samples
- **What it reveals**: Whether cancer and normal samples cluster separately
- **Clinical significance**: Validates that cancer has distinct molecular signatures

**2. Volcano Plot:**
- **Purpose**: Displays both statistical significance and biological magnitude
- **X-axis**: Log2 fold change (magnitude of difference)
- **Y-axis**: -log10 p-value (statistical significance)
- **Clinical significance**: Identifies the most promising therapeutic targets

**3. Heatmap:**
- **Purpose**: Shows expression patterns of top differentially expressed genes
- **Colors**: Red = high expression, Blue = low expression
- **Clinical significance**: Reveals potential biomarkers for cancer diagnosis

**Our visualization script:**
Uses professional ggplot2 graphics to create publication-quality figures suitable for scientific papers and clinical presentations.

In [ ]:
from IPython.display import Image, display
import os

display(Image('heatmap.png'))

Now let's see the annotations of one of these transcripts, e.g. `ENST00000390323.2`:

In [ ]:
%%bash
# Annotations from differentially expressed genes
grep "ENST00000390323.2" refs/22.gtf | cut -f9

---

## 3 Pathway Analysis

### Run Pathway Analysis

**What is pathway enrichment analysis?**
Instead of looking at individual genes, pathway analysis examines groups of genes that work together in biological processes. This helps us understand:

- **Biological mechanisms:** How cancer disrupts normal cellular functions
- **Therapeutic targets:** Which pathways could be targeted with drugs
- **Disease understanding:** The underlying biology of cancer progression

**How does pathway enrichment work?**
1. **Gene sets**: Pre-defined groups of genes involved in specific biological processes
2. **Statistical testing**: Determines if cancer-associated genes are overrepresented in specific pathways
3. **Databases used**:
   - **GO (Gene Ontology)**: Biological processes, molecular functions
   - **KEGG**: Metabolic and signaling pathways
   - **Reactome**: Detailed biological reactions

**Expected cancer pathways:**
- **Upregulated**: Cell cycle, DNA replication, metabolic reprogramming
- **Downregulated**: Apoptosis, DNA repair, immune response

**Clinical relevance:**
Pathway analysis reveals which biological systems are disrupted in cancer, guiding therapeutic strategies and drug development.

In [ ]:
%%bash
Rscript pathway_analysis.R --input degs.tsv --output pathway_results.tsv

In [ ]:
%%bash
head pathway_results.tsv

---

## 4 Drug Repositioning

### Drug Repositioning in Cancer Research

**What is drug repositioning?**
Drug repositioning (also called drug repurposing) involves finding new therapeutic uses for existing drugs. This approach offers several advantages:

1. **Faster development**: 5-10 years vs 15-20 years for new drugs
2. **Known safety profiles**: Existing drugs have established safety data
3. **Lower costs**: Reduces the risk and expense of drug development
4. **Immediate clinical application**: Can be prescribed off-label in some cases

**How does computational drug repositioning work?**
Our approach uses gene expression signatures:
1. **Cancer signature**: Lists of oncogenes (upregulated) and tumor suppressors (downregulated)
2. **Drug effects database**: How thousands of drugs affect gene expression
3. **Signature matching**: Find drugs that reverse cancer gene expression patterns

**Success stories:**
- **Metformin**: Diabetes drug → Cancer prevention (200+ clinical trials)
- **Aspirin**: Pain relief → Cancer prevention (FDA approved)
- **Rapamycin**: Immunosuppressant → Cancer and aging research

**Our script preparation:**
Converts our DESeq2 results into a format compatible with L1000CDS2, a major drug repositioning database.

### Querying the L1000CDS2 Drug Database

**What is L1000CDS2?**
L1000CDS2 (L1000 Characteristic Direction Signature) is a computational tool developed by the Ma'ayan Laboratory that:

1. **Database scope**: Contains gene expression signatures for >20,000 drugs tested on human cell lines
2. **Signature matching**: Uses mathematical algorithms to find drugs that reverse disease signatures
3. **LINCS program**: Part of the NIH Library of Integrated Network-based Cellular Signatures initiative

**How does the algorithm work?**
1. **Input signature**: Our cancer gene signature (oncogenes + tumor suppressors)
2. **Database search**: Compares against drug-induced expression changes
3. **Scoring system**: Calculates how well each drug reverses the cancer signature
4. **Ranking**: Returns drugs ranked by their potential to counteract cancer

**Interpreting the results:**
- **Negative scores**: Drugs that reverse cancer signatures (high therapeutic potential)
- **Positive scores**: Drugs that mimic cancer signatures (avoid these)
- **Score magnitude**: Larger absolute values indicate stronger effects

**Clinical validation:**
The system has successfully identified:
- Known cancer drugs (validates the approach)
- Repositioned drugs already in cancer trials
- Novel repositioning opportunities for further investigation


In [ ]:
%%bash
python drug_repositioning.py --input degs.tsv --output drug_candidates.txt

In [ ]:
%%bash
cat drug_candidates.txt

### Interpretation of Drug Repositioning Results

**Understanding L1000CDS2 Scores:**

The drug repositioning analysis produces a ranked list of compounds based on their ability to reverse cancer gene signatures. Here's how to interpret the results:

**Score Interpretation:**
- **Negative scores**: High therapeutic potential (drugs that reverse cancer signatures)
- **Positive scores**: Avoid these drugs (they mimic or worsen cancer signatures)
- **Score magnitude**: Larger absolute values indicate stronger predicted effects

**Validation Categories:**

**Known Cancer Drugs (Positive Controls):**
- **Examples**: Doxorubicin, Paclitaxel, Cisplatin, Tamoxifen, Imatinib
- **Significance**: Validates our computational approach
- **Clinical meaning**: Confirms the cancer signature is biologically relevant
- **Research value**: Shows the method can identify established cancer therapeutics

**Successfully Repositioned Drugs:**
- **Metformin**: Originally for diabetes → Now in 200+ cancer clinical trials
- **Aspirin**: Originally for pain/inflammation → FDA-approved for cancer prevention
- **Clinical success**: These drugs prove repositioning works in practice
- **Patient benefit**: Already available for off-label use in some cases

**Promising Repositioning Candidates:**
- **Statins** (cholesterol drugs): Anti-cancer properties discovered in studies
- **Rapamycin** (immunosuppressant): Active cancer and aging research
- **Chloroquine** (antimalarial): Being investigated for cancer applications
- **Research opportunity**: Novel applications requiring further validation

---

### Why Drug Repositioning Works

**Scientific advantages:**
- **Faster development timeline**: 5-10 years vs 15-20 years for new drugs
- **Known safety profiles**: Existing drugs have established safety and side effect data
- **Lower development costs**: Reduces financial risk for pharmaceutical companies
- **Regulatory advantages**: Faster approval process for new indications

**Biological rationale:**
- **Pathway targeting**: Many diseases share common molecular pathways
- **Polypharmacology**: Single drugs often affect multiple biological targets
- **Network effects**: Drugs can impact interconnected cellular systems
- **Serendipitous discoveries**: Unexpected beneficial effects in different diseases




---

## Contact

Dr. rer. nat. Israel Barrantes <br>
Research Group Translational Bioinformatics (head)<br>
Institute for Biostatistics and Informatics in Medicine and Ageing Research, Office 3017<br>
Rostock University Medical Center<br>
Ernst-Heydemann-Str. 8<br>
18057 Rostock, Germany<br>

Email: israel.barrantes[bei]uni-rostock.de

---
Last update 2025/10/25
